In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.environ.get('PATH_CUSTOM_MODULES'))

import pandas as pd
import matplotlib.pyplot as plt

In [10]:
path_data = os.environ.get('PATH_DATA_DASHBOARD')
path_aug_exm = os.path.join(path_data, 'augment_sample')

### Refine the scenario guide data

In [11]:
scen_guide = pd.read_csv(os.path.join(path_data, 'scenario_guide.csv'))

scen_guide.color_schema = scen_guide.color_schema.str.upper()
scen_guide.rename(columns={'scenario': 'Skenario',
                            'color_schema': 'Warna',
                            'aug_resize': 'Resize',
                            'aug_rescale': 'Rescale',
                            'aug_horizontal_flip': 'Flip Horizontal',
                            'aug_vertical_flip': 'Flip Vertical',
                            'aug_rotation': 'Rotasi',
                            'aug_brightness': 'Brightness',
                            'aug_contrast_clahe': 'CLAHE',},
                    inplace=True)
scen_guide.replace(to_replace=[1, 0], value=[True, False], inplace=True)
scen_guide.replace(to_replace={'Skenario': True}, value=1, inplace=True)

scen_guide.style.hide()

C:\Users\Bugi\AppData\Local\Temp\ipykernel_19728\821726218.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  scen_guide.replace(to_replace=[1, 0], value=[True, False], inplace=True)
C:\Users\Bugi\AppData\Local\Temp\ipykernel_19728\821726218.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  scen_guide.replace(to_replace={'Skenario': True}, value=1, inplace=True)


Skenario,Warna,Resize,Rescale,Flip Horizontal,Flip Vertical,Rotasi,Brightness,CLAHE
1,RGB,True,True,False,False,False,False,False
2,RGB,True,True,True,True,True,True,False
3,RGB,True,True,True,True,True,True,True


In [12]:
scen_guide.to_csv(os.path.join(path_data, 'rfn_scenario_guide.csv'), index=False)

### Manage the augment example

In [37]:
aug_files = pd.DataFrame({'file_name': os.listdir(path_aug_exm)})

aug_files.drop(aug_files[aug_files.file_name.str.contains('paper')].index, inplace=True)
aug_files['scenario'] = aug_files.file_name.str.split('_').str[1].str.split(' ').str[0]
aug_files['dataset'] = aug_files.file_name.str.split(' ').str[1]
aug_files['augmentation'] = aug_files.file_name.str.split(' ').str[2].str.split('.').str[0]

aug_files.head(5)

,file_name,scenario,dataset,augmentation
8,scenario_1 g1020 no.png,1,g1020,no
9,scenario_1 papila no.png,1,papila,no
10,scenario_1 refuge no.png,1,refuge,no
11,scenario_1 rimone no.png,1,rimone,no
12,scenario_2 g1020 bright.png,2,g1020,bright


In [38]:
aug_files.to_csv(os.path.join(path_data, 'example_aug_imgs.csv'), index=False)

In [46]:
list(aug_files.loc[(aug_files.scenario == '2')
            & (aug_files.augmentation == 'bright'), 'file_name'].values)

['scenario_2 g1020 bright.png',
 'scenario_2 papila bright.png',
 'scenario_2 refuge bright.png',
 'scenario_2 rimone bright.png']

### ---

### Analysis

In [6]:
data_stored = r'./../../../data/evaluation_result/'
data_final = pd.read_csv(os.path.join(data_stored,'merge_result_final.csv'))
data_raw = pd.read_csv(os.path.join(data_stored,'merge_result.csv'))

In [8]:
len(data_final)

72

In [13]:
len(data_final.loc[data_final.scenario == 'scenario_3'])

24